## Create env & run 
The checkpoints are not saved in our training process. The results can be reproduced by running:
>bash run.sh

It might take hours to run the training code. The test results can be found in the folder _./result_

## Method
![Illustration of Our Framework.](./proc.png)

We aim to predict the unpaired probability by estimating the energy parameters in ViennaRNA.

* Reinforcement learning module: As illustrated in the figure, we first extract features from the RNA sequence and the predicted structure by a LSTM module and a GCN module. Then the energy parameters are predicted from the extracted feature vector. The ViennaRNA predict the unpair probability with the given parameters. In the training process, we aim to reduce the RMSD between the ground truth and the predicted unpaired probability. To optimize the parameters in the deep learning model, reinforcement learning strategies are used. In detail, when the energy parameter given by the deep learning model is $p$, the gradients with respect to the model parameters $\theta$ are computed as $\frac{\partial \mathcal{L}((seq, stru, label);\theta)}{\partial \theta}=\frac{RMSD(ViennaRNA(p+\delta_p);label)-RMSD(ViennaRNA(p);label)}{\delta_p}\frac{\partial p}{\partial \theta}$, where the $\delta_p$ is a small value used to estimate the gradient with respect to $p$. In our experiments, we searched $\delta_p$ in {0.01, 0.1, 1.0} and found that $\delta_p=1.0$ performed best.

* Refining module: In practice, we noticed that few RNA structures cannot be accurately predicted by ViennaRNA even with the optimized energy parameters. We compare the predicted probability ($Prob_v$) by ViennaRNA with the binary value ($Prob_b$) given by LinearFold, and for the sequences whose $RMSD(Prob_v, Prob_b)$ is larger than the threshold $\delta$, we refine their predicted probability as $u*Prob_v+(1-u)*Prob_b$.

All the hyperparameters, including the learning rate, $\delta$ and $u$, are optimized by grid search.